# Imports

In [1]:
import numpy as np
import pandas as pd
import numexpr as ne

In [2]:
from simplenn.activations import LRelu
from simplenn.activations import Sigmoid
from simplenn.activations import TanH
from simplenn.loss import Square
from simplenn.loss import NegLogLike
from simplenn.network import Network
from simplenn.network import Layer
from simplenn.optim import BackProp
from simplenn.optim import Genetic

In [3]:
%matplotlib

Using matplotlib backend: TkAgg


# Data

In [4]:
xs = np.array(
    [
        [0,0,0,0,0,0,0,0,0,1],
        [0,0,0,0,0,0,0,0,1,0],
        [0,0,0,0,0,0,0,1,0,0],
        [0,0,0,0,0,0,1,0,0,0],
        [0,0,0,0,0,1,0,0,0,0],
        [0,0,0,0,1,0,0,0,0,0],
        [0,0,0,1,0,0,0,0,0,0],
        [0,0,1,0,0,0,0,0,0,0],
        [0,1,0,0,0,0,0,0,0,0],
        [1,0,0,0,0,0,0,0,0,0]
    ],
    dtype='float64'
).T

ys = np.array(
    [
        [0,0,0,0],
        [0,0,0,1],
        [0,0,1,0],
        [0,0,1,1],
        [0,1,0,0],
        [0,1,0,1],
        [0,1,1,0],
        [0,1,1,1],
        [1,0,0,0],
        [1,0,0,1]
    ],
    dtype='float64'
).T

# Experiments

### Genetic Training

In [9]:
popSize = 50
nElitism = 0
generations = 500
tournamentSize = 5
mutationProba = 0.5
mutationScale = 0.05
mutationRelative = False
verboseFreq = 50
recordFreq = 50

In [18]:
net = Network(
    NegLogLike(),
    [
        Layer(xs.shape[0], 7, TanH(), "kaiming"),
        Layer(7, 7, TanH(), "kaiming"),
        Layer(7, ys.shape[0], Sigmoid(), "kaiming", True)
    ]
)

In [19]:
genetic = Genetic()
net = genetic.run(
    net, 
    xs, 
    ys, 
    popSize, 
    tournamentSize,
    nElitism, 
    mutationProba, 
    mutationScale, 
    mutationRelative, 
    generations, 
    verboseFreq, 
    recordFreq)

Generation 0: 25.494511508272687
Generation 50: 7.698400011827765
Generation 100: 1.8021589160145108
Generation 150: 0.4028176007445713
Generation 200: 0.06759663808859731
Generation 250: 0.01733125808585302
Generation 300: 0.004101470655925241
Generation 350: 0.001271792770187709
Generation 400: 0.0004671678437534354
Generation 450: 0.00016554892222345657


In [20]:
net.forward(xs[:,:9]).round(2)

array([[0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 1., 1., 1., 0.],
       [0., 0., 1., 1., 0., 0., 1., 1., 0.],
       [0., 1., 0., 1., 0., 1., 0., 1., 0.]])

### Gradient Back-Propagation Training

In [21]:
lRate = 0.1
batchSize = xs.shape[1]
epochs = 100*1000
verboseFreq = 10*1000
recordFreq = 10*1000

In [22]:
net = Network(
    NegLogLike(),
    [
        Layer(xs.shape[0], 7, TanH(), "xavier"),
        Layer(7, 7, TanH(), "xavier"),
        Layer(7, 7, TanH(), "xavier"),
        Layer(7, 7, TanH(), "xavier"),
        Layer(7, ys.shape[0], Sigmoid(), "xavier", True)
    ]
)

In [23]:
bp = BackProp()
bp.run(net, xs, ys, batchSize, lRate, epochs, verboseFreq, recordFreq)

Epoch 0: 30.55962935362183
Epoch 10000: 0.031092126281907672
Epoch 20000: 0.015185132675520916
Epoch 30000: 0.010040424675436999
Epoch 40000: 0.007498070007567846
Epoch 50000: 0.00598241444927417
Epoch 60000: 0.0049761098073496084
Epoch 70000: 0.0042593782928694365
Epoch 80000: 0.0037229736162381535
Epoch 90000: 0.003306459258550435


In [24]:
net.forward(xs[:,:9]).round(2)

array([[0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 1., 1., 1., 0.],
       [0., 0., 1., 1., 0., 0., 1., 1., 0.],
       [0., 1., 0., 1., 0., 1., 0., 1., 0.]])

### Network exploration

In [20]:
df = pd.DataFrame({
    f"Layer-{i}":pd.Series([np.abs(n.layers[i].W).sum() for n in bp.networks])
    for i in range(len(genetic.networks[0].layers))
}).plot(title='Weights Norm Evolution')

In [19]:
df = pd.DataFrame({
    f"Layer-{i}":pd.Series([np.abs(n.layers[i].delta_W).sum() for n in bp.networks])
    for i in range(len(bp.networks[0].layers))
}).plot(title='Gradient Norm Evolution')

In [21]:
net = genetic.networks[10]
df = pd.concat(
    [pd.Series(net.layers[i].W.flatten(), name=f"W{i}") for i in range(len(net.layers))],
    axis=1
)
df.hist(bins=20)

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x7f39f0b28a10>,
      dtype=object)

In [22]:
net = bp.networks[10]
df = pd.concat(
    [pd.Series(net.layers[i].A.flatten(), name=f"A{i}") for i in range(len(net.layers))],
    axis=1
)
df.hist(bins=20)

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x7f39f0b2a950>,
      dtype=object)

In [23]:
net = bp.networks[10]
df = pd.concat(
    [pd.Series(net.layers[i].delta_W.flatten(), name=f"delta_W{i}") for i in range(len(net.layers))],
    axis=1
)
df.hist(bins=20)

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x7f39ef050610>,
      dtype=object)

### Numexpr vs. Numpy

In [10]:
%%timeit
A = np.random.random((10000, 5000))
for _ in range(10):
    A = ne.evaluate("cos(A)")

2.46 s ± 183 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%%timeit
A = np.random.random((10000, 5000))
for _ in range(10):
    A = np.cos(A)

6.07 s ± 155 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
